# Main Notebook for Data Generation

This notebook is responsible for generating the data required for analysis and visualization in the results section of the project. The generated data is used in the `Main_document.ipynb` notebook to create figures and perform further analysis.

### Key Points:
- **Problem Formulation:** Problem formulation ID 2 is primarily used for data generation.
- **Random Seeds:** Two main seeds (`20` and `21`) are used to ensure reproducibility. However, a seed of `42` was hardcoded in some parts of the code, which is noted in the comments.
- **Language Model Assistance:** Some parts of the code and comments were generated with the help of large language models like ChatGPT.

### Workflow:
1. Import necessary libraries.
2. Initialize the model using the specified problem formulation.
3. Generate data for various sections, including exploration, sensitivity analysis, optimization, and robustness testing.

In [1]:
import os
import warnings
import re
from datetime import datetime

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import seaborn as sns
from SALib.analyze import sobol


from ema_workbench import MultiprocessingEvaluator, Policy, Scenario, Samplers
from ema_workbench.analysis import dimensional_stacking, parcoords, prim
from ema_workbench.em_framework.optimization import EpsilonProgress, EpsNSGAII
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.util import ema_logging


from problem_formulation import get_model_for_problem_formulation

# Suppress warnings
warnings.filterwarnings("ignore")

# Display all columns in pandas DataFrames
pd.set_option('display.max_columns', None)

# Initialize model
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [ ]:
def save_experiments_and_outcomes(name, experiments, outcomes, folder_name="sobol_results"):
    """
    Saves the experiments and outcomes to CSV files in the specified folder,
    with filenames that include a custom name and timestamp.

    Parameters:
    - name (str): Custom label for the output files (e.g., 'policy6_seed20').
    - experiments (pd.DataFrame): The experiments DataFrame.
    - outcomes (dict): The outcomes dictionary from the EMA Workbench.
    - folder_name (str): The name of the folder where files will be saved. Default is "results".
    """
    # Get timestamp

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Define file paths
    exp_path = os.path.join(folder_name, f'{name}_experiments.csv')
    out_path = os.path.join(folder_name, f'{name}_policies.csv')

    # Save experiments
    experiments.to_csv(exp_path, index=False)

    # Flatten outcomes and save
    flattened_data = {key: value.flatten() for key, value in outcomes.items()}
    df_outcomes = pd.DataFrame(flattened_data)
    df_outcomes.to_csv(out_path, index=False)

    print(f"Saved experiments and outcomes as '{name}' in '{folder_name}')")


## Open Exploration

### Zero Policy

This section generates data for the 'Zero Policy' subsection. The zero policy represents a scenario where no interventions are applied (all levers are set to zero). The data generated here is used to analyze the baseline outcomes without any policy interventions.

### Workflow:
1. Define the number of scenarios to simulate (`n_scenarios = 100000`).
2. Create a policy dictionary where all levers are set to zero.
3. Use the `MultiprocessingEvaluator` to perform experiments with the zero policy across the defined scenarios.
4. Save the generated experiments and outcomes to CSV files for further analysis.




In [ ]:
n_scenarios = 100000

def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "policy 0",
        **dict(
            get_do_nothing_dict()
        )
    ),
]

np.random.seed(20)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies, uncertainty_sampling=Samplers.LHS)

experiments, outcomes = results

folder_name = "zero_policy_results"
experiments.to_csv(os.path.join(folder_name, f'scenario_space_100000_experiments.csv'), index=False)

# Save outcomes DataFrame to a CSV file in the folder
flattened_data = {key: value.flatten() for key, value in outcomes.items()}
df_outcomes = pd.DataFrame(flattened_data)

#df_outcomes = pd.DataFrame(outcomes)
df_outcomes.to_csv(os.path.join(folder_name, f'scenario_space_100000_outcomes.csv'), index=False)

print(f"Experiments and outcomes saved in the '{folder_name}' folder.")

### Subspace Partitioning

This section generates data for the 'Subspace Partitioning' subsection. The goal is to partition the scenario space into smaller subspaces using Latin Hypercube Sampling (LHS). This allows for a more detailed exploration of the scenario and policy spaces.

### Workflow:
1. Define the number of scenarios (`n_scenarios = 600`) and policies (`n_policies = 400`).
2. Use LHS sampling for both uncertainties and levers.
3. Perform experiments using the `MultiprocessingEvaluator`.
4. Save the generated experiments and outcomes to CSV files for further analysis.


In [ ]:
n_scenarios = 600
n_policies = 400

In [ ]:
# running the model to get the LHS data for the subspace partitioning
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=n_policies, uncertainty_sampling=Samplers.LHS, lever_sampling=Samplers.LHS)
experiments, outcomes = results
save_experiments_and_outcomes('LHS_using_pf2', experiments, outcomes, folder_name="LHS_results")

## Sensitivity Analysis (Sobol Indices)

This section generates data for the 'Sobol Indices' subsection. Sobol indices are used to perform sensitivity analysis, which helps identify the most influential uncertainties and levers in the model.

### Workflow:
1. Define a zero policy and a reference scenario with average values for uncertainties.
2. Perform experiments using Sobol sampling for both scenarios and policies.
3. Save the generated experiments and outcomes to CSV files for further analysis.

### Notes:
- The number of scenarios and policies must be a power of two for Sobol sampling.
- The sensitivity analysis helps prioritize uncertainties and levers for optimization.



In [ ]:
# Create a dictionary with all levers set to 0
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

# Create a policy using the do-nothing dictionary
policies = [
    Policy(
        "policy 0",
        **dict(
            get_do_nothing_dict()
        )
    ),]

In [ ]:
# This dataframe is used to show the average values of the uncertainty parameters
df = pd.read_csv("zero_policy_results/scenario_space_100000_experiments.csv")
df.describe()

In [ ]:
# Create a scenario with average values for the uncertainties
reference_values = {
    "Bmax": 190,
    "Brate": 4.167,
    "pfail": 0.5,
    "discount rate 0": 3,
    "discount rate 1": 3,
    "discount rate 2": 3,
    "ID flood wave shape": 4, # Arbritary value, there is no average value for this parameter
}

# Create a scenario dictionary and then a Scenario object
scenario_dict = {}
for key in dike_model.uncertainties:
    name_split = key.name.split("_")
    if len(name_split) == 1:
        scenario_dict[key.name] = reference_values.get(key.name)
    else:
        scenario_dict[key.name] = reference_values.get(name_split[1])
scenario = Scenario("default", **scenario_dict)
scenarios = [scenario,]


Below the experiments can be run, the number of scenarios and policies chosen is very high, as this is needed to perform a sensitivity analysis using Sobol Indices.

In [ ]:
# The amount of scenarios and policies must be a power of two for the Sobol' sequence
n_scenarios = 4096
n_policies = 4096

In [ ]:
# running the model to get the sobol indices for the scenario space
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios, policies=policies, uncertainty_sampling=Samplers.SOBOL)
experiments, outcomes = results
# Save the experiments and outcomes to CSV files
save_experiments_and_outcomes('SOBOL_scenarios', experiments, outcomes, folder_name="sobol_results")

In [ ]:
# running the model to get the sobol indices for the policy space
np.random.seed(21)
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=scenarios, policies=n_policies, lever_sampling=Samplers.SOBOL)   
experiments, outcomes = results
# Save the experiments and outcomes to CSV files
save_experiments_and_outcomes('SOBOL_policies', experiments, outcomes, folder_name="sobol_results")

# Optimization

## Selecting Scenarios for Optimization

This section focuses on selecting scenarios for optimization based on predefined bounds. The selected scenarios represent favorable, medium, and unfavorable conditions.

### Workflow:
1. Load the data generated in the 'Zero Policy' section.
2. Define bounds for favorable, medium, and unfavorable scenarios.
3. Filter and sample scenarios based on the defined bounds.
4. Save the filtered scenarios to CSV files for use in optimization.

### Notes:
- The bounds are defined based on key metrics like expected annual damage and expected number of deaths.
- A hardcoded seed (`42`) was used for sampling, which caused the sampled scenarios to be identical across runs.


In [4]:
# Load the experiments DataFrame
experiments = pd.read_csv(f"zero_policy_results/scenario_space_100000_experiments.csv")
# Load the outcomes DataFrame
outcomes = pd.read_csv(f"zero_policy_results/scenario_space_100000_outcomes.csv")
combined_df = pd.concat([experiments, outcomes], axis=1)


In [5]:
# Define category-specific bounds
favourable_bounds = [
    [[0, 1], [0, 1]]  # 2 scenarios
]

unfavourable_bounds = [
    [[6, float('inf')], [5, float('inf')]]  # 4 scenarios
]

medium_bounds = [
    [[2.5, 3.5], [1.8, 2.8]],  # 2 scenarios
    [[1.5, 2.5], [3, 4]]       # 2 scenarios
]

# Function to filter and sample based on bounds and desired number of samples
def filter_and_sample(bounds_list, combined_df, n_per_bound):
    results = pd.DataFrame()
    for bound in bounds_list:
        mask = (
            (combined_df['Expected Annual Damage'] >= bound[0][0] * 1e9) &
            (combined_df['Expected Annual Damage'] <= bound[0][1] * 1e9 if not np.isinf(bound[0][1]) else True) &
            (combined_df['Expected Number of Deaths'] >= bound[1][0]) &
            (combined_df['Expected Number of Deaths'] <= bound[1][1] if not np.isinf(bound[1][1]) else True)
        )
        filtered = combined_df[mask].copy()
        if not filtered.empty:
            sampled = filtered.sample(n=min(n_per_bound, len(filtered)), random_state=42) # Seed was predefined here :(
            results = pd.concat([results, sampled], ignore_index=True)
        else:
            print(f"No outcomes found for bounds {bound}.")
    return results

# Apply to each category
favourable_scenarios_df = filter_and_sample(favourable_bounds, combined_df, 2).iloc[:, :-39]
medium_scenarios_df = filter_and_sample(medium_bounds, combined_df, 2).iloc[:, :-39]
unfavourable_scenarios_df = filter_and_sample(unfavourable_bounds, combined_df, 4).iloc[:, :-39]

# Save to CSV
favourable_scenarios_df.to_csv("scenarios/favourable_scenarios_df.csv", index=False)
unfavourable_scenarios_df.to_csv("scenarios/unfavourable_scenarios_df.csv", index=False)
medium_scenarios_df.to_csv("scenarios/medium_scenarios_df.csv", index=False)

combined_df = pd.concat([favourable_scenarios_df, medium_scenarios_df, unfavourable_scenarios_df], ignore_index=True)
combined_df.to_csv("scenarios/filtered_scenario_space_df.csv", index=False)



## Running Optimization

This section runs the optimization process for the selected scenarios. The optimization algorithm minimizes multiple objectives, such as expected annual damage and investment costs.

### Workflow:
1. Define the number of function evaluations (`nfe`) and epsilon values for the optimization algorithm.
2. Run the optimization for each scenario using the `EpsNSGAII` algorithm.
3. Save the results (outcomes and convergence metrics) to CSV files for further analysis.

### Notes:
- The optimization process uses the `MultiprocessingEvaluator` for parallel execution.
- The results are used to create Pareto fronts, which represent the trade-offs between objectives.


In [ ]:
def run_optimization(nfe, epsilon, seed, scenario=None):
    ema_logging.log_to_stderr(ema_logging.INFO)
    model, steps = get_model_for_problem_formulation(2)

    #Use default scenario if none is provided
    if scenario is None:
        reference_values = {
            "Bmax": 175,
            "Brate": 1.5,
            "pfail": 0.5,
            "discount rate 0": 3.5,
            "discount rate 1": 3.5,
            "discount rate 2": 3.5,
            "ID flood wave shape": 4,
        }
        scenario_dict = {}
        for key in model.uncertainties:
            name_split = key.name.split("_")
            if len(name_split) == 1:
                scenario_dict[key.name] = reference_values.get(key.name)
            else:
                scenario_dict[key.name] = reference_values.get(name_split[1])
        scenario = Scenario("default", **scenario_dict)

    epsilons = [epsilon] * len(model.outcomes)
    convergence_metrics = [EpsilonProgress()]


    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=nfe,
            searchover="levers",
            algorithm=EpsNSGAII,
            algorithm_kwargs={
                "epsilons": epsilons,
                "problem": scenario,
                "seed": seed, # Seed run 1 = 20, seed run 2 = 21
            },
            convergence=convergence_metrics,
            reference=scenario,
            epsilons=epsilons,
    )

    return results, convergence

def save_results_to_csv(result, name, seed):

    # Define the folder path
    folder_name = f"optimization_150000_seed{seed}_results"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)  # Create the folder if it doesn't exist

    # Save outcomes DataFrame to a CSV file in the folder
    df_result = pd.DataFrame(result)
    df_result.to_csv(os.path.join(folder_name, f'{name}_{current_date}.csv'), index=False)

    print(f"Saved {name}")

In [ ]:
def run_scenarios(unfavourable_scenarios_df, medium_scenarios_df, favourable_scenarios_df, seed, epsilon_value, nfe_value):
    num = 0

    for items in unfavourable_scenarios_df.iterrows():
        print(f"Running unfavourable scenario {num}...")
        scenario_dict = unfavourable_scenarios_df.iloc[num].to_dict()
        scenario = Scenario(f"unfavourable{num}", **scenario_dict)

        np.random.seed(seed)
        result, convergence = run_optimization(epsilon=epsilon_value, nfe=nfe_value, seed=seed,scenario=scenario)

        save_results_to_csv(result, f"unfavourable_outcomes_{nfe_value}_run{seed}_{num}", seed)
        save_results_to_csv(convergence, f"unfavourable_convergence_{nfe_value}_run{seed}_{num}", seed)
        num += 1

    num = 0
    for items in medium_scenarios_df.iterrows():
        print(f"Running medium scenario {num}...")
        scenario_dict = medium_scenarios_df.iloc[num].to_dict()
        scenario = Scenario(f"medium{num}", **scenario_dict)
        result, convergence = run_optimization(epsilon=epsilon_value, nfe=nfe_value, seed=seed, scenario=scenario)

        np.random.seed(seed)
        save_results_to_csv(result, f"medium_outcomes_{nfe_value}_run{seed}_{num}", seed)
        save_results_to_csv(convergence, f"medium_convergence_{nfe_value}_run{seed}_{num}", seed)
        num += 1

    num = 0
    for items in favourable_scenarios_df.iterrows():
        print(f"Running favourable scenario {num}...")
        scenario_dict = favourable_scenarios_df.iloc[num].to_dict()
        scenario = Scenario(f"favourable{num}", **scenario_dict)
        result, convergence = run_optimization(epsilon=epsilon_value, nfe=nfe_value, scenario=scenario)

        np.random.seed(seed)
        save_results_to_csv(result, f"favourable_outcomes_150000_run{seed}_{num}", seed)
        save_results_to_csv(convergence, f"favourable_convergence_150000_run{seed}_{num}", seed)
        num += 1

run_scenarios(unfavourable_scenarios_df, medium_scenarios_df, favourable_scenarios_df, 20, 1000, 150000)
run_scenarios(unfavourable_scenarios_df, medium_scenarios_df, favourable_scenarios_df, 21, 1000, 150000)

## Selecting Policies from Pareto Fronts

This section selects the best policies from the Pareto fronts generated in the optimization process. The selection is based on thresholds and weighted scores.

### Workflow:
1. Define thresholds for key metrics like investment costs and expected annual damage.
2. Filter policies based on the defined thresholds.
3. Select the top policies by minimizing a weighted score that combines multiple robustness metrics.
4. Save the selected policies to CSV files for further analysis.

### Notes:
- The weighted score reflects the importance of different robustness metrics for each outcome.
- The selected policies are tested for robustness in the next section.

In [6]:
def process_scenario_folder(folder):
    """
    Loads and combines CSV files from a folder by scenario type ('favourable', 'medium', 'unfavourable').
    Adds 'Total Investment Costs' and 'Expected Annual Costs' columns, and labels each row with a scenario ID.

    Parameters:
    - folder (str): Path to the folder containing scenario CSV files.

    Returns:
    - favourable_df, unfavourable_df, medium_df (DataFrames): Combined data for each scenario type.
    """
    # === Step 1: List and categorize CSV files ===
    files = os.listdir(folder)
    favourable_files = [f for f in files if f.startswith("favourable_outcomes")]
    unfavourable_files = [f for f in files if f.startswith("unfavourable_outcomes")]
    medium_files = [f for f in files if f.startswith("medium_outcomes")]

    # === Step 2: Define helper to load and combine CSVs ===
    def load_and_combine(file_list, label):
        dfs = []
        for run, file in enumerate(file_list):
            df = pd.read_csv(os.path.join(folder, file))
            df['Total Investment Costs'] = (
                df['Dike Investment Costs'] +
                df['RfR Investment Costs']
            )
            df['Expected Annual Costs'] = (
                df['Evacuation Costs'] +
                df['Expected Annual Damage']
            )
            df['Scenario'] = f"{label}_{run}"
            dfs.append(df)
        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

    # === Step 3: Load and combine each group ===
    favourable_df = load_and_combine(favourable_files, 'Favourable')
    unfavourable_df = load_and_combine(unfavourable_files, 'Unfavourable')
    medium_df = load_and_combine(medium_files, 'Medium')

    return favourable_df, unfavourable_df, medium_df

folder_path = "optimization_150000_seed20_results"
favourable_df_20, unfavourable_df_20, medium_df_20 = process_scenario_folder(folder_path)

folder_path = "optimization_150000_seed21_results"
favourable_df_21, unfavourable_df_21, medium_df_21 = process_scenario_folder(folder_path)


In [ ]:
# Constants for sort priority
SORT_COLUMNS = [
    'Expected Number of Deaths',
    'Expected Annual Damage',
    'Evacuation Costs',
    'Total Investment Costs'
]

# Threshold quantile defaults
DEFAULT_THRESHOLDS = {
    'q_investment_costs': 0.75,
    'q_dike': 0.75,
    'q_evac': 0.50,
    'q_damage': 0.25,
    'death_cap': 0.02
}

def load_thresholds(df, thresholds=DEFAULT_THRESHOLDS):
    df = df.copy()
    df['Total Investment Costs'] = df['Dike Investment Costs'] + df['RfR Investment Costs']
    df['Expected Annual Costs'] = df['Evacuation Costs'] + df['Expected Annual Damage']

    threshold_values = {
        'Total Investment Costs': df['Total Investment Costs'].quantile(thresholds['q_investment_costs']),
        'Dike Investment Costs': df['Dike Investment Costs'].quantile(thresholds['q_dike']),
        'Evacuation Costs': df['Evacuation Costs'].quantile(thresholds['q_evac']),
        'Expected Annual Damage': df['Expected Annual Damage'].quantile(thresholds['q_damage']),
        'Expected Number of Deaths': thresholds['death_cap']
    }

    return df, threshold_values

def filter_df(df, threshold_values):
    return df[
        (df['Total Investment Costs'] <= threshold_values['Total Investment Costs']) &
        (df['Dike Investment Costs'] <= threshold_values['Dike Investment Costs']) &
        (df['Evacuation Costs'] <= threshold_values['Evacuation Costs']) &
        (df['Expected Annual Damage'] <= threshold_values['Expected Annual Damage']) &
        (df['Expected Number of Deaths'] <= threshold_values['Expected Number of Deaths']) &
        (df['RfR Investment Costs'] > 0)
    ]

def select_top_policies(filtered_df, top_n=3):
    return filtered_df.sort_values(by=SORT_COLUMNS).head(top_n)

def save_policy_csv(results_df, prefix, folder_name, save=True):
    if save:
        os.makedirs(folder_name, exist_ok=True)
        filename = os.path.join(folder_name, f"{prefix}policy.csv")
        results_df.to_csv(filename, index=False)
        print(f"Saved: {filename}")

def process_policy_sets(fav_df, med_df, unfav_df, seed):
    fav_loaded, fav_thresholds = load_thresholds(fav_df)
    med_loaded, med_thresholds = load_thresholds(med_df)
    unfav_loaded, unfav_thresholds = load_thresholds(unfav_df)

    fav_selected = filter_df(fav_loaded, fav_thresholds)
    med_selected = filter_df(med_loaded, med_thresholds)
    unfav_selected = filter_df(unfav_loaded, unfav_thresholds)

    # Save filtered dataframes
    save_policy_csv(fav_selected, f"favourable_{seed}_", folder_name=f"policies_seed_{seed}")
    save_policy_csv(med_selected, f"medium_{seed}_", folder_name=f"policies_seed_{seed}")
    save_policy_csv(unfav_selected, f"unfavourable_{seed}_", folder_name=f"policies_seed_{seed}")

    # Top 3 selection from each category
    top3_combined = pd.concat([
        select_top_policies(fav_selected),
        select_top_policies(med_selected),
        select_top_policies(unfav_selected)
    ], ignore_index=True)

    # Save combined top policies
    save_policy_csv(top3_combined, f"top3_combined_{seed}_", folder_name=f"policies_seed_{seed}")

    return top3_combined

# Call the function for both seeds/years
combined_top_policies_20 = process_policy_sets(
    favourable_df_20, medium_df_20, unfavourable_df_20, seed="20"
)

combined_top_policies_21 = process_policy_sets(
    favourable_df_21, medium_df_21, unfavourable_df_21, seed="21"
)


## Evaluating Robustness

This section evaluates the robustness of the selected policies against the set of 10 scenarios. Robustness metrics include mean, standard deviation, mean regret, and max regret.

### Workflow:
1. Load the results of the selected policies.
2. Calculate robustness metrics for each policy.
3. Normalize the metrics to a [0,1] scale.
4. Apply weights to the normalized metrics to calculate a composite score.
5. Select the top policies based on the composite score.

### Notes:
- The robustness evaluation helps identify policies that perform well across a wide range of scenarios.
- The weights reflect the relative importance of different robustness metrics for each outcome.


In [ ]:
# Initialize the dike model and planning steps with formulation version 2
dike_model, planning_steps = get_model_for_problem_formulation(2)

# Load the filtered scenario space CSV file containing scenario parameters
scenario_collection_df = pd.read_csv("scenarios/filtered_scenario_space_df.csv")

# Ignore the last 9 columns which may be metrics or results, keeping only policy parameters

combined_top_policies_df_20 = combined_top_policies_20.iloc[:, :-8]
combined_top_policies_df_21 = combined_top_policies_21.iloc[:, :-8]

np.random.seed(20)

def run_policies_in_scenarios(dike_model, policy_df, scenario_df, seed):
    # Create a timestamped output folder to store results
    output_folder = f"policy_scenario_results_{seed}"
    os.makedirs(output_folder, exist_ok=True)

    # Loop over each policy in the DataFrame
    for i, (idx, policy_row) in enumerate(policy_df.iterrows()):
        print(f"Running policy {i}...")

        # Create an EMA Workbench Policy object from the current policy row
        policy = Policy(f"policy_{i}", **policy_row.to_dict())

        # Create Scenario objects for all scenarios in scenario_df
        scenarios = []
        for j, scenario_row in scenario_df.iterrows():
            scenario = Scenario(f"scenario_{j}", **scenario_row.to_dict())
            scenarios.append(scenario)

        # Use a multiprocessing evaluator to run experiments for the given policy across all scenarios
        with MultiprocessingEvaluator(dike_model) as evaluator:
            experiments, outcomes = evaluator.perform_experiments(
                scenarios=scenarios,
                policies=[policy]
            )

        # Convert experiments (input params) to a DataFrame
        results_df = pd.DataFrame(experiments)

        # Add each outcome as a new column in the results DataFrame
        for outcome_name, outcome_values in outcomes.items():
            results_df[outcome_name] = outcome_values

        # Save results to a CSV file inside the output folder
        filename = os.path.join(output_folder, f"policy_{i}_results.csv")
        results_df.to_csv(filename, index=False)
        print(f"Saved results for policy {i} to {filename}")

# Initialize model and planning steps with formulation version 2
model, steps = get_model_for_problem_formulation(2)

# Run the policies on scenarios
run_policies_in_scenarios(model, combined_top_policies_df_20, scenario_collection_df, 20)
run_policies_in_scenarios(model, combined_top_policies_df_21, scenario_collection_df, 21)



In [ ]:
def evaluate_robustness_with_regret(df, policy_id_col, scenario_id_col, performance_cols, weights):
    # Copy dataframe to avoid modifying original
    regret_df = df.copy()

    # Calculate regret per scenario and performance metric:
    # Regret = (value for policy) - (best value among all policies in that scenario)
    # This tells how much worse a policy performs compared to the best policy in each scenario
    for col in performance_cols:
        regret_df[f'{col} Regret'] = regret_df.groupby(scenario_id_col)[col].transform(lambda x: x - x.min())

    # Group data by policy to compute summary statistics per policy
    grouped = regret_df.groupby(policy_id_col)
    results = []

    for policy, group in grouped:
        metrics = {'Policy': policy}

        # For each performance metric, calculate raw statistics:
        # Mean: average performance over all scenarios
        # Std: variability in performance
        # Max Regret: worst-case regret (how badly it can perform compared to best scenario)
        # Mean Regret: average regret over all scenarios (typical performance gap)
        for col in performance_cols:
            values = group[col]
            regrets = group[f'{col} Regret']

            metrics[f'{col} Mean'] = values.mean()
            metrics[f'{col} Std'] = values.std()
            metrics[f'{col} Max Regret'] = regrets.max()
            metrics[f'{col} Mean Regret'] = regrets.mean()

        results.append(metrics)

    robustness_df = pd.DataFrame(results)

    # --- Normalization Section ---
    # We want to combine metrics that have different scales into a single score.
    # Normalization rescales each metric to a [0,1] range where 0 = best (lowest value),
    # 1 = worst (highest value), so metrics are comparable.
    for col in performance_cols:
        for stat in ['Mean', 'Std', 'Max Regret', 'Mean Regret']:
            metric_col = f'{col} {stat}'

            # Find min and max values of this metric across all policies
            min_val = robustness_df[metric_col].min()
            max_val = robustness_df[metric_col].max()

            # Denominator avoids division by zero if min == max (all values equal)
            denom = max_val - min_val if max_val > min_val else 1e-9

            # Normalize: subtract min then divide by range => scales to 0-1
            robustness_df[f'{metric_col} (Norm)'] = (robustness_df[metric_col] - min_val) / denom

    # --- Weighted Scoring Section ---
    # Apply user-defined weights to normalized metrics reflecting their importance.
    # Weights correspond to your criteria priority for mean, std, max regret, mean regret.
    # Weighted sum gives a single score to rank policies.
    score_cols = []
    for col in performance_cols:
        for stat, weight in zip(['Mean', 'Std', 'Max Regret', 'Mean Regret'], weights[col]):
            norm_col = f'{col} {stat} (Norm)'
            weighted_col = f'{col} {stat} Weighted'
            robustness_df[weighted_col] = robustness_df[norm_col] * weight
            score_cols.append(weighted_col)

    # Sum all weighted metrics to get a final composite score
    # Lower score indicates better overall performance based on priorities
    robustness_df['Total Score'] = robustness_df[score_cols].sum(axis=1)

    return robustness_df

def find_best_policies(folder_path):
    all_dfs = []

    # Load all policy result CSV files from the folder
    for file in os.listdir(folder_path):
        if file.endswith("_results.csv"):
            policy_id = file.replace("_results.csv", "")
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)

            df['policy'] = policy_id
            df['scenario'] = df.index

            # Combine investment costs from different sources
            df['Total Investment Costs'] = df['Dike Investment Costs'] + df['RfR Investment Costs']
            df['Expected Annual Costs'] = df['Evacuation Costs'] + df['Expected Annual Damage']

            all_dfs.append(df)

    if not all_dfs:
        raise ValueError("No result CSV files found.")

    combined_df = pd.concat(all_dfs, ignore_index=True)

    performance_columns = [
        'Expected Number of Deaths',
        'Expected Annual Damage',
        'Evacuation Costs',
        'Total Investment Costs'
    ]

    # Weights according to your priority criteria:
    # - Mean: importance decreases from deaths to investment costs
    # - Std: used for damage, evacuation, investment
    # - Max Regret: only for deaths
    # - Mean Regret: damage, evacuation, investment in descending importance
    weights = {
        'Expected Number of Deaths': [1.0, 1.0, 1.0, 0.01],   # Mean, Std, Max Regret, Mean Regret
        'Expected Annual Damage':   [0.75, 0.75, 0.01, 0.5],
        'Evacuation Costs':         [0.5, 0.5, 0.01, 0.25],
        'Total Investment Costs':   [0.25, 0.25, 0.01, 0.01],
    }

    robustness_df = evaluate_robustness_with_regret(
        combined_df,
        policy_id_col='policy',
        scenario_id_col='scenario',
        performance_cols=performance_columns,
        weights=weights
    )

    # Sort policies by total score ascending (lower is better)
    sorted_df = robustness_df.sort_values('Total Score').reset_index(drop=True)

    # Select top 2 policies based on the weighted score
    best_two = sorted_df.head(2)

    # Print raw performance metrics for the top 2 policies
        # Print raw performance metrics for the top 2 policies
    # Print raw performance metrics for the top 2 policies in a compact tabular format
    for i, row in best_two.iterrows():
        print(f"\nRank {i+1} Policy: {row['Policy']}")
        print(f"{'Metric':30} {'Mean':>15} {'Std':>15} {'Max Regret':>15} {'Mean Regret':>15}")
        print("-" * 90)
        for col in performance_columns:
            print(f"{col:30} "
                  f"{row[f'{col} Mean']:15.6g} "
                  f"{row[f'{col} Std']:15.6g} "
                  f"{row[f'{col} Max Regret']:15.6g} "
                  f"{row[f'{col} Mean Regret']:15.6g}")



    # Return the IDs and full dataframes of the two best policies for further analysis
    best_policy_ids = best_two['Policy'].tolist()
    dfs = [combined_df[combined_df['policy'] == pid] for pid in best_policy_ids]

    return best_policy_ids, dfs


output_folder = "policy_scenario_results_20"
best_policy_ids_20, best_policy_dfs_20 = find_best_policies(output_folder)

output_folder = "policy_scenario_results_21"
best_policy_ids_21, best_policy_dfs_21 = find_best_policies(output_folder)


In [16]:
output_folder = 'selected_4_policies'
os.makedirs(output_folder, exist_ok=True)

# Save policy_8 s20
df_policy_8 = pd.DataFrame(best_policy_dfs_20[0])
file_name_8 = f'policy_8_overview_20.csv'
df_policy_8.to_csv(os.path.join(output_folder, file_name_8), index=False)

# Save policy_6 s20
df_policy_6 = pd.DataFrame(best_policy_dfs_20[1])
file_name_6 = f'policy_6_overview_20.csv'
df_policy_6.to_csv(os.path.join(output_folder, file_name_6), index=False)

# Save policy_8 s21
df_policy_8 = pd.DataFrame(best_policy_dfs_21[0])
file_name_8 = f'policy_8_overview_21.csv'
df_policy_8.to_csv(os.path.join(output_folder, file_name_8), index=False)

# Save policy_7 s21
df_policy_7 = pd.DataFrame(best_policy_dfs_21[1])
file_name_7 = f'policy_7_overview_21.csv'
df_policy_7.to_csv(os.path.join(output_folder, file_name_7), index=False)

## Final Robustness Testing

This section tests the robustness of the final selected policies against 100,000 new scenarios. The goal is to ensure that the policies perform well under diverse conditions.

### Workflow:
1. Load the selected policies and new scenarios.
2. Run experiments for each policy across the new scenarios.
3. Save the results to CSV files for further analysis.

### Notes:
- The robustness testing provides a final validation of the selected policies.
- The results are used to calculate robustness metrics and identify the best-performing policies.


In [11]:
# Define the folder path and get list of files
folder_name = "selected_4_policies"
file_list = os.listdir(folder_name)

# Initialize empty lists to hold dataframes for policies_20 and policies_21
policies_20_dfs = []
policies_21_dfs = []

for file in file_list:
    if file.endswith('.csv'):
        # Read the CSV
        df = pd.read_csv(os.path.join(folder_name, file), nrows=1)

        # Check if file name contains "_20_" or "_21_" and add sliced df accordingly
        if '_20_' in file:
            # Take rows 20 to 50 by index (i.e. iloc[19:50])
            policies_20_dfs.append(df)
        elif '_21_' in file:
            policies_21_dfs.append(df)


policies_20_df = pd.concat(policies_20_dfs, ignore_index=True).iloc[:, 19:50]
policies_21_df = pd.concat(policies_21_dfs, ignore_index=True).iloc[:, 19:50]

policies_20 = []

for idx, row in policies_20_df.iterrows():
    # Convert the row (pandas Series) to a dictionary: lever_name -> lever_value
    lever_values = row.to_dict()

    # Create a unique policy name, e.g. "policy_20_row_0"
    policy_name = f"policy_20_row_{idx}"

    # Create the Policy object
    policy = Policy(policy_name, **lever_values)

    # Append to the list
    policies_20.append(policy)

policies_21 = []

for idx, row in policies_21_df.iterrows():
    # Convert the row (pandas Series) to a dictionary: lever_name -> lever_value
    lever_values = row.to_dict()

    # Create a unique policy name, e.g. "policy_20_row_0"
    policy_name = f"policy_21_row_{idx}"

    # Create the Policy object
    policy = Policy(policy_name, **lever_values)

    # Append to the list
    policies_21.append(policy)

def save_experiments_and_outcomes(name, experiments, outcomes, folder_name="robustness_results"):
    """
    Saves the experiments and outcomes to CSV files in the specified folder,
    with filenames that include a custom name and timestamp.

    Parameters:
    - name (str): Custom label for the output files (e.g., 'policy6_seed20').
    - experiments (pd.DataFrame): The experiments DataFrame.
    - outcomes (dict): The outcomes dictionary from the EMA Workbench.
    - folder_name (str): The name of the folder where files will be saved. Default is "results".
    """
    # Get timestamp
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Ensure folder exists
    os.makedirs(folder_name, exist_ok=True)

    # Define file paths
    exp_path = os.path.join(folder_name, f'{name}_robustness_100000_experiments_{current_date}.csv')
    out_path = os.path.join(folder_name, f'{name}_robustness_100000_outcomes_{current_date}.csv')

    # Save experiments
    experiments.to_csv(exp_path, index=False)

    # Flatten outcomes and save
    flattened_data = {key: value.flatten() for key, value in outcomes.items()}
    df_outcomes = pd.DataFrame(flattened_data)
    df_outcomes.to_csv(out_path, index=False)

    print(f"Saved experiments and outcomes as '{name}' in '{folder_name}' (timestamp: {current_date})")

In [ ]:
n_scenarios = 100000
np.random.seed(21)

def run_policy(name, policy, n_scenarios):
    # Load your model and define parameters
    dike_model, planning_steps = get_model_for_problem_formulation(2)

    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(
            scenarios=n_scenarios,
            policies=policy,
            uncertainty_sampling=Samplers.LHS
        )
    experiments, outcomes = results
    save_experiments_and_outcomes(name, experiments, outcomes)

run_policy('policy_6_s20', policies_20[0], n_scenarios)
run_policy('policy_8_s20', policies_20[1], n_scenarios)
run_policy('policy_7_s21', policies_21[0], n_scenarios)
run_policy('policy_8_s21', policies_21[1], n_scenarios)




### Generating Outcomes for Different Aggregation

Policies are evaluated under a different aggregation level, limited to 50,000 scenarios due to time constraints. The `run_policy` function executes experiments using problem formulation 3, seed 21, performs sampling with LHS, and saves results for four specific policies.

In [ ]:
n_scenarios = 50000
np.random.seed(21)

def run_policy(name, policy, n_scenarios):
    # Load your model and define parameters
    dike_model, planning_steps = get_model_for_problem_formulation(3)

    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(
            scenarios=n_scenarios,
            policies=policy,
            uncertainty_sampling=Samplers.LHS
        )
    experiments, outcomes = results
    save_experiments_and_outcomes(name, experiments, outcomes)

run_policy('policy_6_s20', policies_20[0], n_scenarios)
run_policy('policy_8_s20', policies_20[1], n_scenarios)
run_policy('policy_7_s21', policies_21[0], n_scenarios)
run_policy('policy_8_s21', policies_21[1], n_scenarios)


